In [1]:
from databento import Historical
import pandas as pd
import json

from config import DATABENTO_API_KEY
from utils import split_into_symbol_batches

def write_json(filename, data):
    with open(f'data/{filename}.json', 'w') as f:
        json.dump(data, f, indent=2)

In [2]:
client = Historical(DATABENTO_API_KEY)

In [3]:
pool = pd.read_csv('data/finviz.csv')
pool.drop(columns=['No.', 'P/E', 'Change', 'Volume'], inplace=True)
pool.rename(columns={'Ticker': 'symbol', 'Market Cap': 'mcap'}, inplace=True)
pool.dropna(inplace=True)
pool.columns = [c.lower() for c in pool.columns]
pool

,symbol,company,sector,industry,country,mcap,price
0,AA,Alcoa Corp,Basic Materials,Aluminum,USA,8838.32,34.21
1,AACG,ATA Creativity Global ADR,Consumer Defensive,Education & Training Services,China,25.35,0.79
2,AACT,Ares Acquisition Corporation II,Financial,Shell Companies,USA,688.13,11.01
3,AADI,Aadi Bioscience Inc,Healthcare,Biotechnology,USA,70.83,2.87
4,AAM,AA Mission Acquisition Corp,Financial,Shell Companies,USA,533.36,10.14
...,...,...,...,...,...,...,...
4756,ZVSA,ZyVersa Therapeutics Inc,Healthcare,Biotechnology,USA,3.46,1.38
4757,ZWS,Zurn Elkay Water Solutions Corp,Industrials,Pollution & Treatment Controls,USA,6693.52,39.44
4758,ZYBT,Zhengye Biotechnology Holding Ltd,Healthcare,Drug Manufacturers - Specialty & Generic,China,216.51,4.59
4759,ZYME,Zymeworks BC Inc,Healthcare,Biotechnology,USA,1013.94,14.60


In [4]:
import sqlite3

con = sqlite3.connect('C:\\Users\\ws111\\Desktop\\CODE\\backtest\\data\\data.db')
cur = con.cursor()

pool = []
for row in cur.execute('SELECT DISTINCT(symbol) FROM "ohlcv-1h"'):
    pool.append({'symbol': row[0]})

len(pool)

4612

In [5]:
# batches = split_into_symbol_batches(pool.to_dict(orient='records'))
batches = split_into_symbol_batches(pool)
for b in batches:
    print(len(b), end=' ')
batches

2000 2000 612 

[['CRON',
  'CUK',
  'CGNX',
  'EDIT',
  'AG',
  'AGNC',
  'FR',
  'BW',
  'CLF',
  'HLIT',
  'GNPX',
  'COCP',
  'DBX',
  'CHEK',
  'ATOS',
  'BOX',
  'COMM',
  'DAN',
  'CVLT',
  'BLDP',
  'EAT',
  'FIVN',
  'HBI',
  'EXAS',
  'GERN',
  'CDE',
  'COTY',
  'HAE',
  'HCP',
  'ALK',
  'CAE',
  'ARRY',
  'CETX',
  'AL',
  'GURE',
  'CFR',
  'CUB',
  'BAK',
  'CRSP',
  'FCF',
  'EHTH',
  'FCPT',
  'GOOS',
  'FFWM',
  'CNK',
  'AMRC',
  'GIFI',
  'ARCT',
  'FIZZ',
  'BDJ',
  'HBCP',
  'GNL',
  'ASTE',
  'ACTG',
  'AVDL',
  'BHE',
  'EE',
  'BOH',
  'DGII',
  'CBAN',
  'CHE',
  'EYEN',
  'BBSI',
  'DMRC',
  'FIBK',
  'DM',
  'BSM',
  'CRTO',
  'BDN',
  'ARLP',
  'ETG',
  'FC',
  'EVLV',
  'FRT',
  'ALX',
  'ATXI',
  'GROW',
  'AMRN',
  'FBIZ',
  'EDU',
  'BANF',
  'GNTX',
  'CPS',
  'GSBD',
  'AZUL',
  'ENIC',
  'BFIN',
  'CLRO',
  'CYTK',
  'BGFV',
  'BGH',
  'DENN',
  'CASI',
  'DCO',
  'FRSH',
  'CCOI',
  'BCLI',
  'GBAB',
  'GAIN',
  'ABEO',
  'ETV',
  'FGBI',
  'GLQ',
  'ESE',
  'CTSO'

In [9]:
cost = client.metadata.get_cost('XNAS.ITCH', start='2018-05-01', end='2025-01-31', mode='historical', symbols=batches[2], schema='ohlcv-1d')
cost

1.095271334052

In [14]:
data = client.timeseries.get_range('XNAS.ITCH', start='2018-05-01', end='2025-02-03', symbols=batches[2], schema='ohlcv-1d')
data

C:\Temp\ipykernel_7248\3814970003.py:1: BentoWarning: The streaming request contained one or more days which have reduced quality: 2021-07-07 (degraded), 2021-10-26 (degraded), 2022-09-19 (degraded). See: https://databento.com/docs/api-reference-historical/metadata/metadata-get-dataset-condition
  data = client.timeseries.get_range('XNAS.ITCH', start='2018-05-01', end='2025-02-03', symbols=batches[2], schema='ohlcv-1d')


<DBNStore(schema=ohlcv-1d)>

In [15]:
data.to_csv('data/ohlcv-1d/raw/batch-3.csv', pretty_ts=False)